# Exploratory Data Analisys #

In [ ]:
# Python
from datetime import datetime
import ipywidgets as widgets # Used with date strat/end widgets
import warnings
warnings.filterwarnings('ignore')
import os
import re

# thrid part
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.style.use('seaborn') # ggplot
plt.rcParams['figure.figsize'] = [15, 5]
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 20)
pd.options.display.float_format = '{:,.2f}'.format
import seaborn as sns
sns.set_theme(style="darkgrid")

# Support helper and deep learning functions
from support import Utils as UT
from support import Service as SV

In [ ]:
# Load data

main_dir = 'data'
orders_dir = 'orders'
returns_dir = 'returns'
file_orders = 'Backup_orders_after_etl.csv'
file_models = 'models.xlsx'
file_service = 'service.csv'

try:
    df = UT.load_orders(os.path.join(main_dir, file_orders))
    print('orders: ', df.shape[0])
    df_models = pd.read_excel(os.path.join(main_dir, file_models))
    df_service = pd.read_csv(os.path.join(main_dir, returns_dir, file_service), usecols=['distributor','sku', 'processed'])
    print('returns: ', df_service.shape[0])
except Exception as e:
    print(e)
    
date_start, date_end = UT.show_date()

In [ ]:
UT.plot_pivot_orders(df
                    , df_models
                    , df_service
                    , UT.make_date(date_start)
                    , UT.make_date(date_end)
                    , period_orders = 'M'
                    , df_filter_orders = True # df['Agent'] == 'AVE'
                    , drill_down_orders = ['billing_company'] #only header info, always start with 'Billing Company'
                    , period_lines = 'Q'
                    , df_filter_lines = True # (df['OrderId'].isin(df.loc[df['Billing Company'] == 'Sound&Pixel','OrderId']))
                    , drill_down_lines = ['lineitem_sku'] #['Billing Company','Lineitem sku']
                    , dump = False)

In [ ]:
# Extract different timeseries for the sale channels

year_start = str(UT.make_date(date_start)[0])

channels, df4 = UT.make_pivot_orders_channel(
                    df
                    , period = 'M' # RESAMPLES on a MONTHLY BASIS (no need to interpolate because that's how the original data are)
                    , date_start = (2016,1,1)
                    , date_end = UT.make_date(date_end)
                    , roll = 12
                    )

df4.loc[:,[c + '_cum' for c in channels]].plot(title='Monthly Sales cumulated by sale channel');

df4.loc[year_start, channels].plot(title='Monthly Sales by sale channel');
df4.loc[year_start,[c + '_diff' for c in channels]].plot(title='Monthly Sales difference by sale channel');
df4.loc[year_start,[c + '_roll' for c in channels]].plot(title='Monthly Sales rolling mean by sale channel');

fig = plt.figure()
ax = plt.gca()
for c in channels:
    X = df4[[c]].index.values.astype('float')
    z = np.reshape(np.polyfit(X, df4[[c]], 1), (-1,))
    p = np.poly1d(z)
    ax.plot(X, p(X))
ax.set_title('Monthly Sales Trend by sale channel')
ax.legend(channels)
ax.set_xticklabels('');

# Timeseries for the Total sales

timeseries = df4.Total
timeseries.shape

# Plot mean, variance, and rolling statistics

WINDOW = 3 # MONTHS

# Split the total into chunks
chunks = np.split(timeseries, indices_or_sections=3) 

# Calculate and display mean and variance
mean_vals = np.mean(chunks,axis=1)
var_vals = np.var(chunks,axis=1)
vals = {'mean_vals': mean_vals , 'var_vals': var_vals}
mean_var = pd.DataFrame(vals)

# Plot the data adding resample on a quarterly base and rolling view
fig = plt.figure()
plt.plot(timeseries, 'b')
plt.plot(timeseries.rolling(window=WINDOW).mean(), 'r')
plt.plot(timeseries.rolling(window=WINDOW).std(), 'y')
plt.title('Time series')
plt.legend(['Total by month', 'Rolling mean window = {}'.format(WINDOW), 'Rolling std dev window = {}'.format(WINDOW)])
plt.show()

# Boxplot to show the mean and variance trend
fig = plt.figure()
ax = sns.boxplot(x=timeseries.index.month, y=timeseries)
ax.set_title('Boxplot total sales for month')
plt.grid(b=True)
plt.show()

top_distributors = UT.make_pivot_orders(
        df
        , UT.make_date(date_start)
        , UT.make_date(date_end)
        , period = 'M'
        , df_filter = True
        , drill_down = ['billing_company']  
        , dump = False)
top_distributors.loc[top_distributors.index != 'Total', 'Total']\
.nlargest(10)\
.plot(kind='bar')
plt.show()

top_sku = UT.make_pivot_orderlines(
        df
        , UT.make_date(date_start)
        , UT.make_date(date_end)
        , period = 'M'
        , df_filter = True
        , drill_down = ['lineitem_sku'] 
        , dump = False)
top_sku.loc[top_sku.index != 'Total','Total']\
.nlargest(10)\
.plot(kind='bar')
plt.show()

top_models = UT.make_pivot_orderlines(
        df
        , UT.make_date(date_start)
        , UT.make_date(date_end)
        , period = 'M'
        , df_filter = True
        , drill_down = ['lineitem_model']  
        , dump = False)
top_models.loc[top_models.index != 'Total', 'Total']\
.nlargest(10)\
.plot(kind='bar')
plt.show()